![](problem_statements/E039.png)

# Input

In [1]:
T = int(input().strip())

3


In [2]:
N_list = []
for _ in range(T):
    N_list.append(int(input().strip()))

12
80
120


In [3]:
MAX_P = max(N_list)

# Steps

## Limits on the numbers
For a pythogorean triplet $a,b,c$ we can assume $a<b<c$ without loss of generality. For any given perimeter $p$, these are the limits on $a,b,c$:

$a \in \big[3, p/(2+\sqrt{2}) \big)$ (as the smallest Pythogorean triplet is $\{3,4,5\}$)

$b \in \big[a+1, p/2\big)$

$c \in \big[b+1, p/2\big)$

Considering that $p$ can go upto $\text{MAX_P}$, we can search for $a,b,c$ in the following space:

$a \in \big[3,\text{MAX_P}/(2+\sqrt{2})\big)$

$b \in \big[4,\text{MAX_P}/2\big)$

$c \in [5, \text{MAX_P}/2\big)$

## Euclid's formula
Now, let's use Euclid's formula for primitive Pythogorean triplets

https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple

$a = m^2 - n^2, b = 2mn, c = m^2+n^2$

for $m>n$ and $m,n$ being coprime and not both odd. Eacn pair $m,n$ will create a unique primitve Pyhogorean triple. Let's fid out what limits we can place on $m,n$. Given that $b = 2mn \geq 2n^2$, we can see that

$n \leq \sqrt{\text{MAX_P}}/2$. For his value of $n$, we can search all $m$, ensuring that the perimeter 

$p = a+b+c = 2m(m+n) <= \text{MAX_P}$. So for smallest $n$, we have the largest allowed value of $m$

$m \leq \sqrt{\text{MAX_P}/2}$.

## Co-prime calculation

Let's write a definition to check if two numbers are co-prime. First generate the list of prime numbers.

In [4]:
def get_primes(num):
    "Return a list of all primes <= N"
    
    #Start with the initialization of all numbers from 1 to num as prime=True
    prime = [True if i>=2 else False for i in range(num + 1)]
    
    #Now, run a index from 2 to sqrt(num)
    i = 2
    while i*i<=num:
        #If i is prime, all multiples of i which are <=num will be composite...
        if prime[i]:
            for j in range(i*2,num+1,i):
                prime[j]=False
        i+=1
    #Now, list all the primes - i.e. all n such that prime[n]==True
    ans = []
    for i in range(num+1):
        if prime[i]:
            ans.append(i)
    return ans

In [5]:
primes_list = get_primes(int((MAX_P/2)**0.5)+1)

Now factorize all the allowed values of $m,n$.

In [6]:
from collections import Counter
def get_prime_factors(num):
    ans = Counter()
    n = num
    for p in primes_list: #Note - here we assume that primes_list is the complete list of primes <=sqrt(num)
        if p>int(num**0.5):
            break
        while n%p==0:
            ans[p] += 1
            n//=p
    #If there's a remaining factor...
    if n>1:
        ans[n] += 1
    return ans

In [7]:
prime_factorization = {}
for i in range(1,int((MAX_P/2)**0.5)+1):
    prime_factorization[i] = get_prime_factors(i)

## Generate Pythogorean triplets

First generate primitive triplets.

For each primitive triplet, and it's multiples, use a list to count the number of triplets, with the $p$-th element being the number of triplets with perimeter $p$.

In [8]:
pytho_counter = [0]*(MAX_P+1)

for n in range(1,int(MAX_P**0.5/2)+1):
    for m in filter(lambda x:(n%2==0 or x%2==0) and not bool(prime_factorization[x] & prime_factorization[n]) 
                    and (2*x*(x+n)<=MAX_P),range(n+1,int((MAX_P/2)**0.5)+1)):
        k = 1
        p = 2*m*(m+n)
        
        while(k*p<=MAX_P):
            pytho_counter[k*p] += 1
            k += 1
        #Check that they satisfy the condition...
        #a,b,c = m**2 - n**2,2*m*n,m**2 + n**2
        #assert a**2+b**2==c**2
        #assert a+b+c<=MAX_P
        #assert not(get_prime_factors(a) & get_prime_factors(b))

Now, run along the possible values of $N$ and find the number $<=N$, with the largest value of *pytho_counter*.

In [10]:
solution = [0]*(MAX_P+1)

for k in range(12,MAX_P+1):
    solution[k] = k if pytho_counter[k]>pytho_counter[solution[k-1]] else solution[k-1]

# Output

In [11]:
for N in N_list:
    print(solution[N])

12
60
120
